In [19]:
import pandas as pd
import re

In [20]:
Raw_File = "Final_Recipe.xlsx"
Output_File = "Final_Recipe_Cleaned.xlsx"

In [21]:
Convert_To_List= {
    "keywords":      "keywords_clean",
    "recipeingredientparts": "ingredients",
    "recipeinstructions":    "steps",
}

In [22]:
Tag_Sets = {
    "cuisine": {"north-american","mexican","american","asian","indian",
                "italian","french","chinese","japanese","canadian",
                "hawaiian","south-west-pacific","californian",
                "northeastern-united-states"},
    "dietary": {"vegetarian","vegan","gluten-free","lactose-free",
                "low-sodium","low-carb","low-fat","low-cholesterol",
                "low-protein","low-saturated-fat","free-of-something",
                "healthy","high-calcium"},
    "special": {"easy","kid-friendly","holiday-event","comfort-food",
                "inexpensive","super-bowl","beginner-cook","picnic",
                "copycat","novelty","presentation","dinner-party",
                "taste-mood","to-go","seasonal"},
    "equipment": {"oven","stove-top","deep-fry","grilling","mixer",
                  "food-processor-blender","small-appliance","equipment"},
}

In [23]:

def parse_to_list(val: str) -> list[str]:
    if pd.isna(val):
        return []
    s = str(val).strip()
    s = re.sub(r'^c\(', '', s, flags=re.IGNORECASE).rstrip(')')
    parts = re.split(r'"\s*,\s*"|"\s*,|,\s*', s)
    clean = []
    for p in parts:
        p = p.strip().strip('"\' \u2026.')
        if p:
            clean.append(p.lower())
    return clean

In [24]:
def extract_time(s: str) -> str | None:
    if pd.isna(s): 
        return None
    m = re.search(r'(\d{1,2}:\d{2}(?::\d{2})?)', str(s))
    return m.group(1) if m else None

In [25]:
df = pd.read_excel(Raw_File)
print(f"1️⃣  Loaded raw data: {df.shape}")

1️⃣  Loaded raw data: (522517, 20)


In [27]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r'[^\w]+', '_', regex=True)
)
print("2️⃣  Columns Names : ", list(df.columns))

2️⃣  Columns Names :  ['recipeid', 'name', 'preptime', 'totaltime', 'recipecategory', 'keywords', 'recipeingredientparts', 'aggregatedrating', 'reviewcount', 'calories', 'fatcontent', 'saturatedfatcontent', 'cholesterolcontent', 'sodiumcontent', 'carbohydratecontent', 'fibercontent', 'sugarcontent', 'proteincontent', 'recipeservings', 'recipeinstructions']


In [28]:
before = len(df)
df.dropna(subset=["name","keywords","recipeingredientparts","recipeinstructions"], inplace=True)
print(f"Removed rows with missing importanr values : {before - len(df)}")

Removed rows with missing importanr values : 17237


In [29]:
df["name"] = df["name"].astype(str).str.strip().str.lower()

In [30]:
for raw, clean in Convert_To_List.items():
    df[clean] = df[raw].apply(parse_to_list)
    print(f" Parsed `{raw}` → `{clean}`, sample: {df[clean].iat[0]}")
df.drop(columns=list(Convert_To_List), inplace=True)


 Parsed `keywords` → `keywords_clean`, sample: ['dessert', 'low protein', 'low cholesterol', 'healthy', 'free of', 'summer', 'weeknight', 'freezer', 'easy']
 Parsed `recipeingredientparts` → `ingredients`, sample: ['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']
 Parsed `recipeinstructions` → `steps`, sample: ['toss 2 cups berries with sugar', 'let stand for 45 minutes', 'stirring occasionally', 'transfer berry-sugar mixture to food processor', 'add yogurt and process until smooth', "strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft.  transfer to processor and blend until smooth again", 'return to pan and freeze until edges are solid', 'transfer to processor and blend until smooth again', 'fold in remaining 2 cups of blueberries', 'pour into plastic mold and freeze overnight. let soften slightly to serve']


In [31]:
for cat, tagset in Tag_Sets.items():
    df[cat] = df["keywords_clean"].apply(lambda tags: [t for t in tags if t in tagset])
    uniques = sorted({t for row in df[cat] for t in row})
    print(f"`{cat}` extracted, {len(uniques)} unique sample → {uniques[:5]}")


`cuisine` extracted, 7 unique sample → ['asian', 'canadian', 'chinese', 'hawaiian', 'indian']
`dietary` extracted, 2 unique sample → ['healthy', 'vegan']
`special` extracted, 2 unique sample → ['easy', 'inexpensive']
`equipment` extracted, 2 unique sample → ['mixer', 'oven']


In [32]:
df["num_ingredients"] = df["ingredients"].str.len()
df["num_steps"]       = df["steps"].str.len()
print(" Ingredient counts:", df["num_ingredients"].describe())
print(" Step counts:      ", df["num_steps"].describe())

 Ingredient counts: count    505280.000000
mean          7.879231
std           3.945831
min           1.000000
25%           5.000000
50%           7.000000
75%          10.000000
max          44.000000
Name: num_ingredients, dtype: float64
 Step counts:       count    505280.000000
mean         12.273247
std           8.030155
min           0.000000
25%           7.000000
50%          11.000000
75%          16.000000
max         157.000000
Name: num_steps, dtype: float64


In [33]:
for tcol in ["preptime","totaltime"]:
    if tcol in df:
        before = df[tcol].notna().sum()
        df[tcol] = df[tcol].apply(extract_time)
        after = df[tcol].notna().sum()
        print(f" `{tcol}` cleaned: dropped {before-after} invalid entries")


 `preptime` cleaned: dropped 0 invalid entries
 `totaltime` cleaned: dropped 0 invalid entries


In [34]:
rename_map = {
    "recipeid":          "recipe_id",
    "recipecategory":    "recipe_category",
    "aggregatedrating":  "rating",
    "reviewcount":       "review_count",
    "recipeservings":    "recipe_servings",
    "fatcontent":        "fat_content",
    "saturatedfatcontent":"saturated_fat_content",
    "cholesterolcontent":"cholesterol_content",
    "sodiumcontent":     "sodium_content",
    "carbohydratecontent":"carbohydrate_content",
    "fibercontent":      "fiber_content",
    "sugarcontent":      "sugar_content",
    "proteincontent":    "protein_content",
}
df.rename(columns=rename_map, inplace=True)
for col in ["rating","review_count"]:
    if col in df:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        print(f" `{col}` has {df[col].isna().sum()} NaNs after numeric")

 `rating` has 242321 NaNs after numeric
 `review_count` has 236771 NaNs after numeric


In [35]:
before = len(df)
df.drop_duplicates(subset=["name"], inplace=True)
print(f"Duplicates dropped: {before - len(df)}")

Duplicates dropped: 81873


In [36]:
df.reset_index(drop=True, inplace=True)
df.columns = df.columns.str.replace('_', ' ').str.title()
print(f"Final cleaned shape: {df.shape}")
df.to_excel(Output_File, index=False)
print(f"Saved cleaned data to {Output_File}")


Final cleaned shape: (423407, 26)
Saved cleaned data to Final_Recipe_Cleaned.xlsx
